<div style="padding:16px 18px;border:1px solid rgba(0,0,0,.10);border-radius:16px;background:#ffffff;">
  <div style="font-size:28px;font-weight:800;line-height:1.15;">
    Approaching_DS · <span style="color:#009BC8;">Basis: Connecting a free LLM service</span>
  </div>
  <div style="margin-top:6px;color:rgba(0,0,0,.65);">
    Subtítulo: Se muestra una conexión a LLM open - source.
  </div>

  <div style="height:4px;background:#009BC8;border-radius:999px;margin:14px 0 12px 0;"></div>

  <table style="width:100%;border-collapse:collapse;">
    <tr>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Autor</b><br><span style="color:rgba(0,0,0,.65);">@romarc98</span>
      </td>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Fecha</b><br><span style="color:rgba(0,0,0,.65);">2026-Q1</span>
      </td>
      <td style="padding:8px 10px;border:1px solid rgba(0,0,0,.08);border-radius:12px;">
        <b>Status</b><br><span style="color:rgba(0,0,0,.65);">Draft</span>
      </td>
    </tr>
  </table>
  </div>
</div>


<div style="padding:10px 12px;border-radius:14px;border:1px solid rgba(0,155,200,.35);background:rgba(0,155,200,.10);">
  <b style="color:#007FA6;">Info</b><br>
  <span style="color:rgba(0,0,0,.70);">
    Experimentación con el Client de Ollama y la capacidad para llamar a sus "n" modelos open - source.
  
  </span>
</div>


In [1]:
import requests

def check_ollama():
    try:
        r = requests.get("http://localhost:11434/api/version", timeout=2)
        return r.status_code, r.text
    except Exception as e:
        return None, str(e)

check_ollama()


(200, '{"version":"0.13.5"}')

In [2]:
import requests, json, time

BASE = "http://127.0.0.1:11434"

def pull_model(model: str, timeout_s: int = 3600):
    """
    Descarga un modelo en Ollama usando la API HTTP.
    Muestra progreso aproximado si el servidor lo reporta.
    """
    url = f"{BASE}/api/pull"
    payload = {"model": model, "stream": True}

    with requests.post(url, json=payload, stream=True, timeout=timeout_s) as r:
        r.raise_for_status()

        last_print = time.time()
        for line in r.iter_lines():
            if not line:
                continue
            msg = json.loads(line.decode("utf-8"))

            status = msg.get("status", "")
            total = msg.get("total", 0)
            completed = msg.get("completed", 0)

            # Throttle prints
            if time.time() - last_print < 0.05:
                continue
            last_print = time.time()

            if total:
                pct = (completed / total) * 100
                print(f"{model} | {status:20s} | {pct:6.2f}%  ({completed/1e6:,.1f} / {total/1e6:,.1f} MB)", end="\r")
            else:
                # Mensajes sin barra de progreso
                print(f"{model} | {status:20s} | {msg}", end="\r")

    print("\nDescarga finalizada.")

# Ejemplo: modelo ligero para empezar
pull_model("llama3.2")

# pull_model("llama3.1:8b")
# pull_model("qwen2.5:7b-instruct")
# pull_model("mistral:7b-instruct")
# pull_model("gemma2:2b")

llama3.2 | pulling dde5aa3fc5ff | 100.00%  (2,019.4 / 2,019.4 MB)
Descarga finalizada.


In [3]:
import requests, json

BASE = "http://127.0.0.1:11434"  # usa 127.0.0.1 para evitar rarezas con localhost en Windows

def hit(method, path, payload=None, timeout=15):
    url = f"{BASE}{path}"
    try:
        r = requests.request(method, url, json=payload, timeout=timeout)
        ct = r.headers.get("content-type", "")
        text = r.text[:500]
        print(f"{method} {path} -> {r.status_code} | {ct}")
        if text:
            print(text)
        print("-"*80)
        return r
    except Exception as e:
        print(f"{method} {path} -> EXCEPTION: {e}")
        print("-"*80)
        return None

# 1) Nativo
hit("GET", "/api/version")
hit("GET", "/api/tags")

GET /api/version -> 200 | application/json; charset=utf-8
{"version":"0.13.5"}
--------------------------------------------------------------------------------
GET /api/tags -> 200 | application/json; charset=utf-8
{"models":[{"name":"llama3.2:latest","model":"llama3.2:latest","modified_at":"2026-01-07T17:48:35.1496098+01:00","size":2019393189,"digest":"a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"3.2B","quantization_level":"Q4_K_M"}}]}
--------------------------------------------------------------------------------


<Response [200]>

In [4]:
import requests

BASE = "http://127.0.0.1:11434"

r = requests.get(f"{BASE}/v1/models", timeout=30)
r.raise_for_status()

models = [m["id"] for m in r.json().get("data", [])]
models

['llama3.2:latest']

In [5]:
import requests

BASE = "http://127.0.0.1:11434"
MODEL = models[0]

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "Responde en español, técnico y conciso."},
        {"role": "user", "content": "Qué frameworks existen actuamente para gestionar flujos agénticos? (cita almenos 4)"}
    ],
    "temperature": 0.2,
    "stream": False
}

r = requests.post(f"{BASE}/v1/chat/completions", json=payload, timeout=120)
r.raise_for_status()

j = r.json()
print("MODEL:", MODEL)
print(j["choices"][0]["message"]["content"])


MODEL: llama3.2:latest
Existen varios frameworks para gestionar flujos de agentes, que se enfocan en la modelización y el análisis de sistemas complejos. A continuación, te menciono algunos de los más destacados:

1. **Framework de Flujos de Agentes (AFL)**: Desarrollado por el Instituto Nacional de Investigación en Ciencias Aplicadas y Tecnología Industrial (INCIPI), este framework se enfoca en la modelización de sistemas complejos utilizando flujos de agentes.
2. **Framework de Modelado de Flujos de Agentes (MFSA)**: Propuesto por la Universidad Politécnica de Madrid, este framework utiliza técnicas de modelado para analizar y simular sistemas complejos que involucran flujos de agentes.
3. **Framework de Análisis de Flujos de Agentes (AFS)**: Desarrollado por el Instituto de Investigación en Sistemas Complejos (IISC), este framework se enfoca en la análisis y la evaluación de sistemas complejos que involucran flujos de agentes.
4. **Framework de Modelado de Redes de Agentes (MRA)**: 

In [6]:
# Modelo de visión:
pull_model("moondream")

moondream | writing manifest     | {'status': 'writing manifest'}
Descarga finalizada.
